## Open Policy Agent

In [1]:
# import required packages (install if needed)
using OpenPolicyAgent
using OpenAPI

import OpenPolicyAgent: ASTWalker
import OpenPolicyAgent.ASTWalker.AST: ASTVisitor
import OpenPolicyAgent.ASTWalker.SQL: SQLVisitor, UnconditionalInclude, UnconditionalExclude

In [2]:
# define policy (rego)
const policy = """
    package accounting_system

    # no one is allowed by default
    default allow = false
    default check_business_hours = false

    check_business_hours {
        input.hourofday >= 9
        input.hourofday <= 17
        input.dayofweek >= 1
        input.dayofweek <= 5
    }

    # allow all users during business hours when audit is not happening
    allow {
        input.is_audit_time == false
        input.is_logged_in == true
        check_business_hours
    }

    # allow boss at all hours when audit is not happening
    allow {
        input.is_audit_time == false
        input.is_logged_in == true
        input.user.role == "boss"
    }

    # allow only auditors during business hours when audit is happening
    allow {
        input.is_audit_time == true
        input.is_logged_in == true
        input.user.role == "auditor"
        check_business_hours
    }
""";

In [3]:
# Start the OPA server
opa_server = OpenPolicyAgent.Server.MonitoredOPAServer("demo_config.yaml")
OpenPolicyAgent.Server.start!(opa_server);

In [4]:
# Create client to interact with the OPA server
openapi_client = OpenAPI.Clients.Client("http://localhost:8181"; escape_path_params=false)
policy_client = OpenPolicyAgent.Client.PolicyApi(openapi_client)
compile_client = OpenPolicyAgent.Client.CompileApi(openapi_client);

In [5]:
# Initialize the server by loading the policy
# Note: This is a demo.
#    Dynamic loading of policies must be done securely.
#    Policy servers can be configured with policies directly.
response, _ = OpenPolicyAgent.Client.put_policy_module(policy_client, "accounting_system", policy)
@assert isa(response, OpenPolicyAgent.Client.PutPolicySuccessResponse);

In [6]:
function is_allowed(input)
    query = "data.accounting_system.allow == true"
    unknowns = haskey(input, "user") ? [] : ["input.user"]
    partial_query_schema = OpenPolicyAgent.Client.PartialQuerySchema(;
        query = query,
        input = input,
        unknowns = unknowns,
    )

    # compile the query by partially evaluating it
    response, _ = OpenPolicyAgent.Client.post_compile(
        compile_client;
        partial_query_schema = partial_query_schema,
    )
    @assert isa(response, OpenPolicyAgent.Client.CompileSuccessResponse)

    # translate resultant AST to SQL
    result = response.result
    ast = ASTWalker.walk(ASTVisitor(), result)
    sqlvisitor = SQLVisitor(Dict{String, String}(), Dict{String, String}())
    sqlcondition = ASTWalker.walk(sqlvisitor, ast)

    return isa(sqlcondition, UnconditionalExclude) ? "false" :
           isa(sqlcondition, UnconditionalInclude) ? "true" :
           sqlcondition.sql
end

is_allowed (generic function with 1 method)

In [7]:
# users are allowed during business hours, when there's no audit
is_allowed(Dict(
        "hourofday" => 10,
        "dayofweek" => 2,
        "is_audit_time" => false,
        "is_logged_in" => true,
        "user" => Dict("role" => "user")
    )
)

"true"

In [8]:
# users are not allowed outside business hours
is_allowed(Dict(
        "hourofday" => 1,
        "dayofweek" => 2,
        "is_audit_time" => false,
        "is_logged_in" => true,
        "user" => Dict("role" => "user")
    )
)

"false"

In [9]:
# boss is allowed at all hours, when there's no audit
is_allowed(Dict(
        "hourofday" => 1,
        "dayofweek" => 2,
        "is_audit_time" => false,
        "is_logged_in" => true,
        "user" => Dict("role" => "boss")
    )
)

"true"

In [10]:
# even boss is not allowed when there's an audit
is_allowed(Dict(
        "hourofday" => 10,
        "dayofweek" => 2,
        "is_audit_time" => true,
        "is_logged_in" => true,
        "user" => Dict("role" => "boss")
    )
)

"false"

In [11]:
# find the list of users who are allowed access during audit
is_allowed(Dict(
        "hourofday" => 10,
        "dayofweek" => 2,
        "is_audit_time" => true,
        "is_logged_in" => true,
    )
)

"( input.user.role = 'auditor' )"

In [12]:
# find list of users who are allowed access outside business hours
is_allowed(Dict(
        "hourofday" => 1,
        "dayofweek" => 2,
        "is_audit_time" => false,
        "is_logged_in" => true,
    )
)

"( input.user.role = 'boss' )"

In [13]:
# Stop the OPA server
OpenPolicyAgent.Server.stop!(opa_server)